In [1]:
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import pandas_ta as ta
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from plotly.subplots import make_subplots

from algrow import Algrow, TrainingData




### Get data

In [2]:
btc = TrainingData(symbol="BTCAUD", bar="15m", start_time="January 01, 2023")
btc.get_data()

In [3]:
eth = TrainingData(symbol="ETHAUD", bar="1h", start_time="January 01, 2020")
eth.get_data()

In [4]:
ada = TrainingData(symbol="ADAAUD", bar="1h", start_time="January 01, 2020")
ada.get_data()

In [5]:
sol = TrainingData(symbol="SOLAUD", bar="1h", start_time="January 01, 2020")
sol.get_data()

In [6]:
bnb = TrainingData(symbol="BNBAUD", bar="1h", start_time="January 01, 2020")
bnb.get_data()

### Add indicators + find a good predictive feature

In [3]:
# Indicators

# Predictive feature
pred_sma = 3
shift = -3

indicators = ta.Strategy(
    name="Momo and Volatility",
    description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
    ta=[
        {"kind": "sma", "length": 200},
        {"kind": "sma", "length": 50},
        {"kind": "sma", "length": 20},
        {"kind": "sma", "length": 5},
        {"kind": "sma", "length": 8},
        {"kind": "sma", "length": pred_sma},
        {"kind": "sma", "length": 3},
        {"kind": "rsi", "length": 14},
        {"kind": "bbands", "length": 20},
        {"kind": "macd", "fast": 8, "slow": 21, "col_names": ("MACD", "MACD_H", "MACD_S")},
        {"kind": "stochrsi", "length": 14}
        ]
    )

In [4]:
btc.add_indicators(indicators)
btc.data["shift"] = btc.data[f'SMA_{pred_sma}'].shift(shift)
btc.data['close_diff'] = ((btc.data["shift"] - btc.data[f'SMA_{pred_sma}'])/btc.data[f'SMA_{pred_sma}'])*100
btc.data.dropna(inplace=True)


In [9]:
eth.add_indicators(indicators)
eth.data["shift"] = eth.data[f'SMA_{pred_sma}'].shift(shift)
eth.data['close_diff'] = ((eth.data["shift"] - eth.data[f'SMA_{pred_sma}'])/eth.data[f'SMA_{pred_sma}'])*100
eth.data.dropna(inplace=True)


In [10]:
sol.add_indicators(indicators)
sol.data["shift"] = sol.data[f'SMA_{pred_sma}'].shift(shift)
sol.data['close_diff'] = ((sol.data["shift"] - sol.data[f'SMA_{pred_sma}'])/sol.data[f'SMA_{pred_sma}'])*100
sol.data.dropna(inplace=True)

In [11]:
bnb.add_indicators(indicators)
bnb.data["shift"] = bnb.data[f'SMA_{pred_sma}'].shift(shift)
bnb.data['close_diff'] = ((bnb.data["shift"] - bnb.data[f'SMA_{pred_sma}'])/bnb.data[f'SMA_{pred_sma}'])*100
bnb.data.dropna(inplace=True)

In [12]:
ada.add_indicators(indicators)
ada.data["shift"] = ada.data[f'SMA_{pred_sma}'].shift(shift)
ada.data['close_diff'] = ((ada.data["shift"] - ada.data[f'SMA_{pred_sma}'])/ada.data[f'SMA_{pred_sma}'])*100
ada.data.dropna(inplace=True)

In [6]:
# Check predictive feature is useful
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01, row_heights=[0.6, 0.4])

bars = 1500 
# CANDLE STICK CHART
fig.add_trace(go.Candlestick(x=btc.data['closetime'][-bars:],
                open=btc.data['open'][-bars:],
                high=btc.data['high'][-bars:],
                low=btc.data['low'][-bars:],
                close=btc.data['close'][-bars:]), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data["shift"][-bars:],
                         opacity=0.7,
                         line=dict(color='orange', width=2),
                         name=f'SMA_{pred_sma}_{shift}'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data[f'SMA_{pred_sma}'][-bars:],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name=f'SMA_{pred_sma}'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:], y=btc.data['close_diff'][-bars:],
                    mode='lines',
                    name='close_diff',
                    line=dict(color='black',width=3)), row=2, col=1)
                    


fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Predictive Feature")
fig.show()


print(eth.data.columns)

NameError: name 'eth' is not defined

In [14]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing = 0.01, row_heights=[0.7, 0.2, 0.2, 0.2, 0.3])

bars = 300

# CANDLE STICK CHART
fig.add_trace(go.Candlestick(x=btc.data['closetime'][-bars:],
                open=btc.data['open'][-bars:],
                high=btc.data['high'][-bars:],
                low=btc.data['low'][-bars:],
                close=btc.data['close'][-bars:]), row=1, col=1)

# MOVING AVERAGES
fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['SMA_200'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='EMA_200'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['SMA_50'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_50'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['SMA_20'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_20'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['SMA_3'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_3'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['SMA_5'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_5'), row=1, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['SMA_8'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_8'), row=1, col=1)



##BBANDS

fig.add_trace(go.Scatter(x = btc.data['closetime'][-bars:],
                         y = btc.data['BBU_20_2.0'][-bars:],
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         name = 'BBU_20_2.0',
                         opacity = 0.2),
              row = 1, col = 1)

# Lower Bound fill in between with parameter 'fill': 'tonexty'
fig.add_trace(go.Scatter(x = btc.data['closetime'][-bars:],
                         y = btc.data['BBL_20_2.0'][-bars:],
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'BBL_20_2.0',
                         opacity = 0.2),
              row = 1, col = 1)

fig.add_trace(go.Scatter(x = btc.data['closetime'][-bars:],
                         y = btc.data['BBM_20_2.0'][-bars:],
                         line_color = 'red',
                         line = {'dash': 'dash'},
                         name = 'BBM_20_2.0',
                         opacity = 0.5),
              row = 1, col = 1)




#RSI

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['RSI_14'][-bars:],
                         opacity=0.7,
                         line=dict(color='black', width=2),
                         name='RSI_14'), row=2, col=1)



# STOCH RSI

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['STOCHRSIk_14_14_3_3'][-bars:],
                         opacity=0.7,
                         line=dict(color='black', width=2),
                         name='STOCHRSIk_14_14_3_3'), row=3, col=1)

fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['STOCHRSId_14_14_3_3'][-bars:],
                         opacity=0.7,
                         line=dict(color='red', width=2),
                         name='STOCHRSId_14_14_3_3'), row=3, col=1)


## BB std

fig.add_trace(go.Scatter(x = btc.data['closetime'][-bars:],
                         y = btc.data['BBB_20_2.0'][-bars:],
                         line_color = 'red',
                         line = {'dash': 'dash'},
                         name = 'BBB_20_2.0',
                         opacity = 0.5),
              row = 4, col = 1)

fig.add_trace(go.Scatter(x = btc.data['closetime'][-bars:],
                         y = btc.data['BBP_20_2.0'][-bars:],
                         line_color = 'red',
                         line = {'dash': 'dash'},
                         name = 'BBP_20_2.0',
                         opacity = 0.5),
              row = 4, col = 1)




# MACD
fig.add_trace(go.Bar(x=btc.data['closetime'][-bars:],
                     y=btc.data['MACD_H'][-bars:]
                    ), row=5, col=1)
fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['MACD'][-bars:],
                         line=dict(color='black', width=2)
                        ), row=5, col=1)
fig.add_trace(go.Scatter(x=btc.data['closetime'][-bars:],
                         y=btc.data['MACD_S'][-bars:],
                         line=dict(color='blue', width=1)
                        ), row=5, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Input Features")
fig.show()

In [15]:
btc_features = btc.data[[
    'open', 'high', 'low', 'close', 'volume', 'ntrades', 
    'SMA_200', 'SMA_50', 'SMA_20', 'SMA_8', 'SMA_5', 'SMA_3', 
    'MACD', 'MACD_H', 'MACD_S', 
    'STOCHRSId_14_14_3_3', 'STOCHRSIk_14_14_3_3', 
    'BBB_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'BBL_20_2.0', 'BBL_20_2.0',
    'RSI_14',
    'close_diff']]

In [16]:
eth_features = eth.data[[
    'open', 'high', 'low', 'close', 'volume', 'ntrades', 
    'SMA_200', 'SMA_50', 'SMA_20', 'SMA_8', 'SMA_5', 'SMA_3', 
    'MACD', 'MACD_H', 'MACD_S', 
    'STOCHRSId_14_14_3_3', 'STOCHRSIk_14_14_3_3', 
    'BBB_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'BBL_20_2.0', 'BBL_20_2.0',
    'RSI_14',
    'close_diff']]


In [17]:
ada_features = ada.data[[
    'open', 'high', 'low', 'close', 'volume', 'ntrades', 
    'SMA_200', 'SMA_50', 'SMA_20', 'SMA_8', 'SMA_5', 'SMA_3', 
    'MACD', 'MACD_H', 'MACD_S', 
    'STOCHRSId_14_14_3_3', 'STOCHRSIk_14_14_3_3', 
    'BBB_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'BBL_20_2.0', 'BBL_20_2.0',
    'RSI_14',
    'close_diff']]

In [18]:
sol_features = sol.data[[
    'open', 'high', 'low', 'close', 'volume', 'ntrades', 
    'SMA_200', 'SMA_50', 'SMA_20', 'SMA_8', 'SMA_5', 'SMA_3', 
    'MACD', 'MACD_H', 'MACD_S', 
    'STOCHRSId_14_14_3_3', 'STOCHRSIk_14_14_3_3', 
    'BBB_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'BBL_20_2.0', 'BBL_20_2.0',
    'RSI_14',
    'close_diff']]

In [19]:
bnb_features = bnb.data[[
    'open', 'high', 'low', 'close', 'volume', 'ntrades', 
    'SMA_200', 'SMA_50', 'SMA_20', 'SMA_8', 'SMA_5', 'SMA_3', 
    'MACD', 'MACD_H', 'MACD_S', 
    'STOCHRSId_14_14_3_3', 'STOCHRSIk_14_14_3_3', 
    'BBB_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'BBL_20_2.0', 'BBL_20_2.0',
    'RSI_14',
    'close_diff']]

In [20]:
btc_sc = MinMaxScaler(feature_range=(0,1))
btc_scaled_features = btc_sc.fit_transform(btc_features)
print(np.shape(btc_scaled_features))

(21861, 24)


In [21]:
eth_sc = MinMaxScaler(feature_range=(0,1))
eth_scaled_features = eth_sc.fit_transform(eth_features)
print(np.shape(eth_scaled_features))

(21861, 24)


In [22]:
ada_sc = MinMaxScaler(feature_range=(0,1))
ada_scaled_features = ada_sc.fit_transform(ada_features)
print(np.shape(ada_scaled_features))

(16660, 24)


In [23]:
bnb_sc = MinMaxScaler(feature_range=(0,1))
bnb_scaled_features = bnb_sc.fit_transform(bnb_features)
print(np.shape(bnb_scaled_features))

(21701, 24)


In [24]:
sol_sc = MinMaxScaler(feature_range=(0,1))
sol_scaled_features = sol_sc.fit_transform(sol_features)
print(np.shape(sol_scaled_features))

(12302, 24)


In [25]:
btc_target_min = btc_features["close_diff"].min()
btc_target_max = btc_features["close_diff"].max()
btc_close_min = btc_features["close"].min()
btc_close_max = btc_features["close"].max()
btc_open_min = btc_features["open"].min()
btc_open_max = btc_features["open"].max()
btc_low_min = btc_features["low"].min()
btc_low_max = btc_features["low"].max()
btc_high_min = btc_features["high"].min()
btc_high_max = btc_features["high"].max()
(btc_target_min, btc_target_max)

(-9.113210539322104, 11.981750560592552)

### Reshape data from X, Y

In [26]:
# multiple feature from data provided to the model
btc_X = []

backcandles = 24
num_features = 23


for j in range(num_features):
    btc_X.append([])
    for i in range(backcandles, btc_scaled_features.shape[0]):
        btc_X[j].append(btc_scaled_features[i-backcandles:i, j])

#move axis from 0 to position 2
btc_X=np.moveaxis(btc_X, [0], [2])

btc_X, btc_yi =np.array(btc_X), np.array(btc_scaled_features[backcandles:,-1:])
btc_y=np.reshape(btc_yi,(len(btc_yi),1))

print(btc_X.shape)
print(btc_y.shape)

(21837, 24, 23)
(21837, 1)


In [27]:
# multiple feature from data provided to the model
eth_X = []

for j in range(num_features):
    eth_X.append([])
    for i in range(backcandles, eth_scaled_features.shape[0]):
        eth_X[j].append(eth_scaled_features[i-backcandles:i, j])

#move axis from 0 to position 2
eth_X=np.moveaxis(eth_X, [0], [2])

eth_X, eth_yi =np.array(eth_X), np.array(eth_scaled_features[backcandles:,-1:])
eth_y=np.reshape(eth_yi,(len(eth_yi),1))

print(eth_X.shape)
print(eth_y.shape)

(21837, 24, 23)
(21837, 1)


In [28]:
# multiple feature from data provided to the model
ada_X = []


for j in range(num_features):
    ada_X.append([])
    for i in range(backcandles, ada_scaled_features.shape[0]):
        ada_X[j].append(ada_scaled_features[i-backcandles:i, j])

#move axis from 0 to position 2
ada_X=np.moveaxis(ada_X, [0], [2])

ada_X, ada_yi =np.array(ada_X), np.array(ada_scaled_features[backcandles:,-1:])
ada_y=np.reshape(ada_yi,(len(ada_yi),1))

print(ada_X.shape)
print(ada_y.shape)

(16636, 24, 23)
(16636, 1)


In [29]:
# multiple feature from data provided to the model
bnb_X = []


for j in range(num_features):
    bnb_X.append([])
    for i in range(backcandles, bnb_scaled_features.shape[0]):
        bnb_X[j].append(bnb_scaled_features[i-backcandles:i, j])

#move axis from 0 to position 2
bnb_X=np.moveaxis(bnb_X, [0], [2])

bnb_X, bnb_yi =np.array(bnb_X), np.array(bnb_scaled_features[backcandles:,-1:])
bnb_y=np.reshape(bnb_yi,(len(bnb_yi),1))

print(bnb_X.shape)
print(bnb_y.shape)

(21677, 24, 23)
(21677, 1)


In [30]:
# multiple feature from data provided to the model
sol_X = []


for j in range(num_features):
    sol_X.append([])
    for i in range(backcandles, sol_scaled_features.shape[0]):
        sol_X[j].append(sol_scaled_features[i-backcandles:i, j])

#move axis from 0 to position 2
sol_X=np.moveaxis(sol_X, [0], [2])

sol_X, sol_yi =np.array(sol_X), np.array(sol_scaled_features[backcandles:,-1:])
sol_y=np.reshape(sol_yi,(len(sol_yi),1))

print(sol_X.shape)
print(sol_y.shape)

(12278, 24, 23)
(12278, 1)


In [31]:
# split data into train test sets
splitlimit = int(len(btc_X)*0.9)
btc_x_train, btc_x_test = btc_X[:splitlimit], btc_X[splitlimit:]
btc_y_train, btc_y_test = btc_y[:splitlimit], btc_y[splitlimit:]
print(btc_x_train.shape)
print(btc_x_test.shape)
print(btc_y_train.shape)
print(btc_y_test.shape)

(19653, 24, 23)
(2184, 24, 23)
(19653, 1)
(2184, 1)


In [32]:
eth_x_train = eth_X
eth_y_train = eth_y
print(eth_x_train.shape)
print(eth_y_train.shape)

(21837, 24, 23)
(21837, 1)


In [33]:
ada_x_train = ada_X
ada_y_train = ada_y
print(ada_x_train.shape)
print(ada_y_train.shape)

(16636, 24, 23)
(16636, 1)


In [34]:
bnb_x_train = bnb_X
bnb_y_train = bnb_y
print(bnb_x_train.shape)
print(bnb_y_train.shape)

(21677, 24, 23)
(21677, 1)


In [35]:
sol_x_train = sol_X
sol_y_train = sol_y
print(sol_x_train.shape)
print(sol_y_train.shape)

(12278, 24, 23)
(12278, 1)


In [36]:
x_train = np.concatenate((ada_x_train, eth_x_train, sol_x_train, bnb_x_train, btc_x_train))
y_train = np.concatenate((ada_y_train, eth_y_train, sol_y_train, bnb_y_train, btc_y_train))
x_test = btc_x_test
y_test = btc_y_test
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(92081, 24, 23)
(92081, 1)
(2184, 24, 23)
(2184, 1)


In [37]:
x_train_input = torch.tensor(x_train).to(torch.float32)
y_train_input = torch.tensor(y_train).to(torch.float32)

x_test_input = torch.tensor(x_test).to(torch.float32)
y_test_input = torch.tensor(y_test).to(torch.float32)

ds_train = TensorDataset(x_train_input, y_train_input)
ds_test = TensorDataset(x_test_input, y_test_input)

In [38]:
batch_size=32
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

In [39]:
# Check it's working
for batch, (X, y) in enumerate(dl_train):
    print(f"Batch: {batch+1}")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")

    break


"""
Batch: 1
X shape: torch.Size([64, 2])
y shape: torch.Size([64])
"""

Batch: 1
X shape: torch.Size([32, 24, 23])
y shape: torch.Size([32, 1])


'\nBatch: 1\nX shape: torch.Size([64, 2])\ny shape: torch.Size([64])\n'

In [40]:
num_epochs = 60 #1000 epochs
learning_rate = 0.001 #0.001 lr
look_back = backcandles
input_size = num_features #number of features
hidden_size = num_features #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
pred_size = 1

model = Algrow(input_size, hidden_size, look_back, num_layers, pred_size = pred_size, drop_out=0.3)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

loss = []
val_loss = []

def train():

    train_loss = 0
    validation_loss = 0

    for i, (x, y) in enumerate(dl_train):
        model.train()
        optimizer.zero_grad()
        pred_y = model(x)
        real_y = y
        loss = criterion(pred_y, real_y)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    for j, (x, y) in enumerate(dl_test):
        val_pred = model(x)
        val_loss = criterion(val_pred, y)
        validation_loss += val_loss.item()

    return (train_loss/i, validation_loss/j)


for e in range(num_epochs):
    train_loss, validation_loss = train()
    loss.append(train_loss)
    val_loss.append(validation_loss)
    if e%2==0:
        print(f'Epoch {e},{train_loss:.5e}, {validation_loss:.5e}')
  

/Users/alex/opt/anaconda3/envs/algrow/lib/python3.8/site-packages/torch/nn/modules/rnn.py:67: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1



Epoch 0,4.10441e-03, 2.13449e-03
Epoch 2,2.76989e-03, 2.12077e-03
Epoch 4,2.55484e-03, 1.69570e-03
Epoch 6,2.41392e-03, 1.79268e-03
Epoch 8,2.33798e-03, 1.63140e-03
Epoch 10,2.26740e-03, 1.66431e-03
Epoch 12,2.20798e-03, 1.50062e-03
Epoch 14,2.15782e-03, 2.04575e-03
Epoch 16,2.11835e-03, 1.76230e-03
Epoch 18,2.08380e-03, 1.92155e-03
Epoch 20,2.04526e-03, 1.86195e-03
Epoch 22,2.01027e-03, 1.68913e-03
Epoch 24,1.99038e-03, 1.91746e-03
Epoch 26,1.95351e-03, 1.94755e-03
Epoch 28,1.93003e-03, 1.82774e-03
Epoch 30,1.89727e-03, 1.89811e-03
Epoch 32,1.87567e-03, 1.92423e-03
Epoch 34,1.84220e-03, 1.86632e-03
Epoch 36,1.83600e-03, 1.88212e-03
Epoch 38,1.82114e-03, 2.13180e-03
Epoch 40,1.79657e-03, 2.06659e-03
Epoch 42,1.77665e-03, 1.92889e-03
Epoch 44,1.76139e-03, 2.12238e-03
Epoch 46,1.74331e-03, 2.23666e-03
Epoch 48,1.74086e-03, 1.97123e-03
Epoch 50,1.72361e-03, 1.67022e-03
Epoch 52,1.70966e-03, 1.89035e-03
Epoch 54,1.68487e-03, 2.09446e-03
Epoch 56,1.68777e-03, 2.01556e-03
Epoch 58,1.66296e-0

In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(loss))-1, y=loss[1:],
                    mode='lines+markers',
                    name='Train Loss',
                    line=dict(color="blue",width=1)))

fig.add_trace(go.Scatter(x=np.arange(len(val_loss))-1, y=val_loss[1:],
                    mode='lines+markers',
                    name='Validation Loss',
                    line=dict(color="red",width=1)))

fig.update_layout(template="plotly_dark")

In [42]:
test_open_unscaled = x_test[:, -1, 0]*(btc_open_max-btc_open_min)+btc_open_min
test_high_unscaled = x_test[:, -1, 1]*(btc_high_max-btc_high_min)+btc_high_min
test_low_unscaled = x_test[:, -1, 2]*(btc_low_max-btc_low_min)+btc_low_min
test_close_unscaled = x_test[:, -1, 3]*(btc_close_max-btc_close_min)+btc_close_min

train_open_unscaled = x_train[:, -1, 0]*(btc_open_max-btc_open_min)+btc_open_min
train_high_unscaled = x_train[:, -1, 1]*(btc_high_max-btc_high_min)+btc_high_min
train_low_unscaled = x_train[:, -1, 2]*(btc_low_max-btc_low_min)+btc_low_min
train_close_unscaled = x_train[:, -1, 3]*(btc_close_max-btc_close_min)+btc_close_min

In [43]:
y_pred = model(x_train_input)


y_pred_unscaled = y_pred.detach()[:].flatten()*(btc_target_max-btc_target_min)+btc_target_min
y_train_unscaled = y_train[:].flatten()*(btc_target_max-btc_target_min)+btc_target_min

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01, row_heights=[0.7, 0.2])

bars = 500

# CANDLE STICK CHART
fig.add_trace(go.Candlestick(x=np.arange(np.shape(train_open_unscaled)[0])[-bars:],
                open=train_open_unscaled[-bars:],
                high=train_high_unscaled[-bars:],
                low=train_low_unscaled[-bars:],
                close=train_close_unscaled[-bars:]), row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(np.shape(y_train_unscaled)[0])[-bars:], y=y_train_unscaled[-bars:],
                    mode='lines',
                    name='BTC truth',
                    line=dict(color='black',width=3)), row=2, col=1)
                    
fig.add_trace(go.Scatter(x=np.arange(y_pred_unscaled.shape[0])[-bars:], y=y_pred_unscaled[-bars:],
                    mode='lines',
                    name='BTC pred',
                    line=dict(color='red',width=3),
                    opacity=0.5), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Training")
fig.show()

#fig.update_layout(template="plotly_dark")

In [44]:
y_pred = model(x_test_input)


y_pred_unscaled = y_pred.detach()[:].flatten()*(btc_target_max-btc_target_min)+btc_target_min
y_test_unscaled = y_test[:].flatten()*(btc_target_max-btc_target_min)+btc_target_min

bars = 500

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01, row_heights=[0.7, 0.2])

# CANDLE STICK CHART
fig.add_trace(go.Candlestick(x=np.arange(np.shape(test_open_unscaled)[0])[-bars:],
                open=test_open_unscaled[-bars:],
                high=test_high_unscaled[-bars:],
                low=test_low_unscaled[-bars:],
                close=test_close_unscaled[-bars:]), row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(np.shape(y_test_unscaled)[0])[-bars:], y=y_test_unscaled[-bars:],
                    mode='lines',
                    name='BTC truth',
                    line=dict(color='black',width=3)), row=2, col=1)
                    
fig.add_trace(go.Scatter(x=np.arange(y_pred_unscaled.shape[0])[-bars:], y=y_pred_unscaled[-bars:],
                    mode='lines',
                    name='BTC pred',
                    line=dict(color='red',width=3),
                    opacity=0.5), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Validation")
fig.show()

#fig.update_layout(template="plotly_dark")